# Dataset Collection and Indexing

In [1]:
CORPUS_PATH = "collections/msmarco-passage"
COLLECTION = "trec-deep-learning-passages"
COLLECTION_ZIP_PATH = "collections/msmarco-passage/collectionandqueries.tar.gz"
import pyterrier as pt


In [2]:
if not pt.started():
  pt.init(mem=8000)


PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [16]:

dataset = pt.get_dataset("trec-deep-learning-passages")
def msmarco_generate():
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

# Single threaded indexing           
# iter_indexer = pt.IterDictIndexer("./passage_index")
# indexref3 = iter_indexer.index(msmarco_generate(), meta=['docno', 'text'], meta_lengths=[20, 4096])            

# Multi threaded indexing, UNIX-based systems only!!!!!           
iter_indexer = pt.IterDictIndexer("./passage_index_8", threads=8)
indexref4 = iter_indexer.index(msmarco_generate(), meta=['docno', 'text'], meta_lengths=[20, 4096])

14:23:22.496 [ForkJoinPool-1-worker-5] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields


/Users/danielvangelder/miniconda3/lib/python3.8/site-packages/pyterrier/index.py:551: UserWarning: Using multiple threads results in a non-deterministic ordering of document in the index. For deterministic behavior, use threads=1
  warn('Using multiple threads results in a non-deterministic ordering of document in the index. For deterministic behavior, use threads=1')


14:23:22.505 [ForkJoinPool-1-worker-3] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.570 [ForkJoinPool-1-worker-13] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.665 [ForkJoinPool-1-worker-1] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.670 [ForkJoinPool-1-worker-9] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.672 [ForkJoinPool-1-worker-15] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.689 [ForkJoinPool-1-worker-11] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.690 [ForkJoinPool-1-worker-7] INFO  o.t.structures.indexing.Indexer - Indexer using 1 fields
14:23:22.723 [ForkJoinPool-1-worker-11] INFO  o.t.structures.indexing.Indexer - creating the data structures data_stream2_1
14:23:22.723 [ForkJoinPool-1-worker-15] INFO  o.t.structures.indexing.Indexer - creating the data structures data_stream5_1
14:23:22.723 [ForkJoinPool-1-worker

14:29:26.727 [ForkJoinPool-1-worker-9] INFO  o.t.structures.indexing.Indexer - Started building the inverted index...
14:29:26.908 [ForkJoinPool-1-worker-7] INFO  o.t.structures.indexing.Indexer - Started building the inverted index...
14:29:26.924 [ForkJoinPool-1-worker-11] INFO  o.t.s.i.c.InvertedIndexBuilder - BasicMemSizeLexiconScanner: lexicon scanning until approx 5.7 GiB of memory is consumed
14:29:26.924 [ForkJoinPool-1-worker-1] INFO  o.t.s.i.c.InvertedIndexBuilder - BasicMemSizeLexiconScanner: lexicon scanning until approx 5.7 GiB of memory is consumed
14:29:26.924 [ForkJoinPool-1-worker-9] INFO  o.t.s.i.c.InvertedIndexBuilder - BasicMemSizeLexiconScanner: lexicon scanning until approx 5.7 GiB of memory is consumed
14:29:27.270 [ForkJoinPool-1-worker-7] INFO  o.t.s.i.c.InvertedIndexBuilder - BasicMemSizeLexiconScanner: lexicon scanning until approx 5.7 GiB of memory is consumed
14:29:27.271 [ForkJoinPool-1-worker-3] INFO  o.t.structures.indexing.Indexer - Started building the

14:30:14.581 [ForkJoinPool-1-worker-7] INFO  o.t.s.indexing.LexiconBuilder - Optimising structure lexicon
14:30:14.582 [ForkJoinPool-1-worker-9] INFO  o.t.s.merging.StructureMerger - merged inverted files in 9.802
14:30:14.586 [ForkJoinPool-1-worker-13] INFO  o.t.s.indexing.LexiconBuilder - lexicon has 1 fields
14:30:14.590 [ForkJoinPool-1-worker-7] INFO  o.t.s.indexing.LexiconBuilder - lexicon has 1 fields
14:30:14.594 [ForkJoinPool-1-worker-9] INFO  o.t.s.indexing.LexiconBuilder - Optimising structure lexicon
14:30:14.602 [ForkJoinPool-1-worker-13] INFO  o.t.s.i.FSOMapFileLexiconUtilities - Optimising lexicon with 554862 entries
14:30:14.611 [ForkJoinPool-1-worker-7] INFO  o.t.s.i.FSOMapFileLexiconUtilities - Optimising lexicon with 555066 entries
14:30:14.617 [ForkJoinPool-1-worker-9] INFO  o.t.s.indexing.LexiconBuilder - lexicon has 1 fields
14:30:14.642 [ForkJoinPool-1-worker-9] INFO  o.t.s.i.FSOMapFileLexiconUtilities - Optimising lexicon with 553702 entries
14:30:15.043 [ForkJoi

In [17]:
pt.logging('WARN')
index = pt.IndexFactory.of(indexref4)
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



# Feature Extraction

In [15]:
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:BM25","WMODEL:Tf", "WMODEL:PL2"])

# Learning to rank with LambdaMART

In [ ]:
import lightgbm as lgb
# this configures LightGBM as LambdaMART lmart_l = lgb.LGBMRanker(task="train",
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=100,
    max_bin=255,
    num_leaves=7,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[1, 3, 5, 10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=10)
lmart_l_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_l, form="ltr")
lmart_l_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)
pt.Experiment(
    [bm25, lmart_x_pipe, lmart_l_pipe],
    test_topics,
    test_qrels,
    ["map"],
    names=["BM25 Baseline", "LambdaMART (xgBoost)", "LambdaMART (LightGBM)" ]
)